# Measurement notebook example

The following tutorials all focus on a specific aspect of arbok. This notebook on the other hand gives as few comments as possible and provides structure of a real world measurement notebook.

## 1. Importing modules

### 1.1 Import of arbok modules

In [1]:
from arbok_driver import (
    ArbokDriver,
    Sample,
    Sequence,
    create_measurement_loop,
    utils
)

2024-04-09 11:23:16,874 - qm - INFO     - Starting session: 68bf9f60-58db-42f5-8cff-490fe1610efb


### 1.2 Import self written sequences and configurations

In [2]:
from example_sequences.square_pulse import SquarePulse
from example_sequences.dummy_readout import DummyReadout

In [3]:
from example_configs.square_pulse_config import square_pulse_conf
from example_configs.dummy_readout_config import dummy_readout_config
from example_configs.dummy_sample import dummy_sample

### 1.3 Generic imports

In [23]:
import numpy as np
import os
%load_ext rich

## 2. Configuring the driver

### 2.1 Instanciating driver, sequence and set relations

In [4]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)
dummy_sequnece = Sequence('dummy_sequnece', dummy_sample)
qm_driver.add_sequence(dummy_sequnece)

In [5]:
square_pulse = SquarePulse('square_pulse', dummy_sample, square_pulse_conf)
readout = DummyReadout('readout', dummy_sample, dummy_readout_config)
dummy_sequnece.add_subsequence(square_pulse)
dummy_sequnece.add_subsequence(readout)

dict_keys(['readout_qubit1__ref__sensor1_I', 'readout_qubit1__ref__sensor1_Q', 'readout_qubit1__ref__sensor1_IQ'])
dict_keys(['readout_qubit1__read__sensor1_I', 'readout_qubit1__read__sensor1_Q', 'readout_qubit1__read__sensor1_IQ'])


### 2.2 Set sweep parameters and values

In [7]:
dummy_sequnece.set_sweeps(
    {
        square_pulse.amplitude: np.linspace(0.1, 1, 5)
    },
    {
        square_pulse.t_square_pulse: np.arange(20, 100, 10, dtype = int)
    }, 
)

### 2.3 Register the gettables to save

Print gettables of a `ReadSequence` to see all available gettables.

In [26]:
# readout.gettables

In [18]:
dummy_sequnece.register_gettables(
    readout.qubit1.diff(),
    readout.qubit1.read.sensor1_IQ(),
    readout.qubit1.ref.sensor1_IQ(),
)

### 2.4 Compile QUA program and print to file

In [ ]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    file_name = '0_qua_program.py', 
    qua_program = qua_program,
    add_config=False)

### 2.5 Connect QM and run program

In [ ]:
IP_ADDRESS = '<YOUR_QM_IP>'
qm_driver.connect_opx(IP_ADDRESS)
qm_driver.run(qua_program)

In [ ]:
sweep_list = [
    {qm_driver.iteration: np.arange(100)},
]

## 3. Set up QCoDeS datamanagement and run measurement 

### 3.1 Define qcodes database path

In [ ]:
db_file_path = os.path.join(os.getcwd(), 'example.db')
initialise_or_create_database_at(db_file_path)

### 3.2 Define the experiment and measurement

In [25]:
MEASUREMENT_NAME = "specific measurement name"

In [ ]:
tutorial_exp = load_or_create_experiment(
    experiment_name="Example Experiment",
    sample_name="dummy_sample"
)
meas = Measurement(exp = tutorial_exp, name = MEASUREMENT_NAME)

### 3.3 Create measurement loop and run the experiment

In [ ]:
@create_measurement_loop(
    sequence = dummy_sequnece, measurement=meas, sweep_list=sweep_list)
def run_measurement_loop():
    pass

In [ ]:
run_measurement_loop()